In [1]:
import sys
from pathlib import Path
# Add src directory to path to import organoid_analysis package
sys.path.insert(0, str(Path('..') / 'src'))
import napari
import pandas as pd
from cellpose import models, core, io
import pyclesperanto_prototype as cle
from tifffile import imread 
from skimage.measure import regionprops_table
from organoid_analysis.utils import list_images, read_image, extract_scaling_metadata, remap_labels, segment_organoids_from_cp_labels, extract_organoid_stats_and_merge

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose models
model = models.CellposeModel(gpu=True, model_type="cyto3") 

2026-02-17 15:43:09,066 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2026-02-17 15:43:09,066 [INFO] 
cellpose version: 	3.1.1.3 
platform:       	win32 
python version: 	3.10.19 
torch version:  	2.5.0
2026-02-17 15:43:09,117 [INFO] ** TORCH CUDA version installed and working. **
2026-02-17 15:43:09,118 [INFO] >> cyto3 << model set to be used
2026-02-17 15:43:09,118 [INFO] ** TORCH CUDA version installed and working. **
2026-02-17 15:43:09,119 [INFO] >>>> using GPU (CUDA)
2026-02-17 15:43:09,152 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cyto3
2026-02-17 15:43:09,242 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [2]:
# Define the markers you wish to analyze and its cellular compartment (i.e. cell,membrane or nuclei)
# ("channel_name", position, location)
markers = [("Occludin_RFP", 0, "membrane"), ("Claudin_FITC", 1, "membrane"), ("Occludin_RFP", 0, "cell"), ("DAPI", 2, "nuclei")]

In [3]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\B2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\B6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\C2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\C6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\D2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\D6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\E2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\P

In [4]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id from filename
well_id = filename.split("_")[0]

# Extract x,y,z scaling from .nd2 file metadata in order feed the Z pixel size / XY pixel size ratio into Cellpose
pixel_size_x, pixel_size_y, voxel_size_z = extract_scaling_metadata(image)

assert pixel_size_x == pixel_size_y
z_to_xy_ratio = voxel_size_z / pixel_size_x
print(z_to_xy_ratio)


Image analyzed: B2
Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 2.000 µm
3.017451031446116


In [5]:
# Type down the multiposition index you want to explore
position = 0

# (suffix, subdir, channels, cellprob_threshold) for each compartment
# channels: [cyto_chan, nuclear_chan] for cell; [nuclear_chan, grayscale] for nuclei (indices in cellpose_input 0=mem, 1=nuclei, 2=Cellmask)
segment_configs = [
    ("cell", "cellpose_cell_labels", [3, 2]),   # Cellmask (2), nuclei (1) in cellpose_input
    ("nuclei", "cellpose_nuclei_labels", [2, 0]),  # nuclei (1), grayscale
]

print(f"Analyzing multiposition index {position}")

# Open one of the multipositions in the img file
single_img = img[position]
# Input shape for next step has to be C, Z, Y, X
single_img = single_img.transpose(1, 0, 2, 3)

# Keep only 3 meaningful input fluorescence channels to mimick Cellpose GUI normalization
# 0 membrane marker, 2 nuclei, 3 Cellmask (Numpy 0-index) → cellpose_input: 0=mem, 1=nuclei, 2=Cellmask
cellpose_input = single_img[[0, 2, 3], :, :, :]

labels = {}
for suffix, subdir, channels in segment_configs:
    cellpose_filename = f"{well_id}_{position}_{suffix}"
    prediction_path = directory_path / subdir / f"{cellpose_filename}.tif"

    if prediction_path.exists():
        print(f"Predictions already calculated for: {cellpose_filename}")
        labels[suffix] = imread(prediction_path)
    else:
        labels[suffix], _, _ = model.eval(
            cellpose_input,
            channels=channels,
            diameter=20,
            do_3D=True,
            anisotropy=z_to_xy_ratio,
            normalize=True,
            flow_threshold=0.4,
            cellprob_threshold=0.0,
            min_size=15,
        )

del cellpose_input

cytoplasm_labels = labels["cell"]
nuclei_labels = labels["nuclei"]

# Remap nuclei labels to match their surrounding cytoplasm id
nuclei_labels = remap_labels(nuclei_labels, cytoplasm_labels)

# Initialize Napari Viewer
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(single_img)
viewer.add_labels(cytoplasm_labels)
viewer.add_labels(nuclei_labels)

Analyzing multiposition index 0
2026-02-17 15:43:45,451 [INFO] multi-stack tiff read in as having 21 planes 3 channels
2026-02-17 15:43:45,682 [INFO] resizing 3D image with rescale=1.50 and anisotropy=3.017451031446116
2026-02-17 15:43:46,924 [INFO] running YX: 95 planes of size (2040, 2040)
2026-02-17 15:44:22,017 [INFO] 100%|##########| 95/95 [00:35<00:00,  2.71it/s]
2026-02-17 15:44:24,570 [INFO] running ZY: 2040 planes of size (95, 2040)
2026-02-17 15:45:00,681 [INFO] 100%|##########| 2040/2040 [00:36<00:00, 56.49it/s]
2026-02-17 15:45:02,321 [INFO] running ZX: 2040 planes of size (95, 2040)
2026-02-17 15:45:48,529 [INFO] 100%|##########| 2040/2040 [00:46<00:00, 44.15it/s]
2026-02-17 15:45:53,979 [INFO] resizing 3D flows and cellprob to original image size
2026-02-17 15:45:55,167 [INFO] network run in 129.49s
2026-02-17 15:45:57,295 [INFO] masks created in 1.97s
2026-02-17 15:45:59,161 [INFO] multi-stack tiff read in as having 21 planes 3 channels
2026-02-17 15:45:59,503 [INFO] res

<Labels layer 'nuclei_labels' at 0x235cd611750>

In [6]:
# Create a dictionary containing all image descriptors
#TODO: Add multiposition index during BP
descriptor_dict = {
            "filename": filename,
            "well_id": well_id,
            }

props_list = []

# Single list of regionprops to request; rename_map is built from this below
regionprops_properties = [
    "label",
    "area",
    "intensity_mean",
    "intensity_min",
    "intensity_max",
    "intensity_std",
]

membrane_labels = None # Variable to check in order to compute membrane_labels just once, avoid repeated GPU ops

# Loop through markers and extract 
for marker_name, ch_nr, location in markers:
    print(f"Analyzing channel: {marker_name} in {location} ...")

    # Region cell, nuclei or membrane: generate membrane and extract info from that location if needed.
    # Pick label image and optionally create membrane layer.
    if location == "cell":
        label_image = cytoplasm_labels
    elif location == "nuclei":
        label_image = nuclei_labels
    elif location == "membrane":
        layer_name = "membrane_labels"
        # Check if layer already exists in the viewer
        if layer_name not in viewer.layers:
            # Generate membrane if needed
            if membrane_labels is None:
                membrane_labels = cle.reduce_labels_to_label_edges(cytoplasm_labels)
                membrane_labels = cle.pull(membrane_labels)
            viewer.add_labels(membrane_labels, name=layer_name)
            label_image = membrane_labels
        else:
            # Layer already present → use its data
            label_image = viewer.layers[layer_name].data
            if membrane_labels is None:
                membrane_labels = label_image
    else:
        raise ValueError(f"Unknown location: {location}")

    props = regionprops_table(
        label_image=label_image,
        intensity_image=single_img[ch_nr],
        properties=regionprops_properties,
    )

    # Convert to dataframe
    props_df = pd.DataFrame(props)

    # Rename columns to indicate the specific image; build rename_map from regionprops_properties:
    # label unchanged; area -> location_area; intensity_* -> prefix_*_int (concentration proxy etc.)
    prefix = f"{location}_{marker_name}"
    rename_map = {"label": "label"}
    for prop in regionprops_properties:
        if prop == "label":
            continue
        elif prop == "area":
            rename_map[prop] = f"{location}_area"
        elif prop.startswith("intensity_"):
            suffix = prop.replace("intensity_", "")
            rename_map[prop] = f"{prefix}_{suffix}_int"

    props_df.rename(columns=rename_map, inplace=True)

    # Derived columns (use names from rename_map so they stay in sync with regionprops_properties)
    mean_col = rename_map["intensity_mean"]
    max_col = rename_map["intensity_max"]
    area_col = rename_map["area"]
    # Max / mean ratio (puncta vs diffuse signal)
    props_df[f"{prefix}_max_mean_ratio"] = props_df[max_col] / props_df[mean_col]
    # Total marker content per cell
    props_df[f"{prefix}_sum_int"] = props_df[mean_col] * props_df[area_col]

    # Append each props_df to props_list
    props_list.append(props_df)

# Initialize the df with the first df in the list
props_df = props_list[0]
# Start looping from the second df in the list
for df in props_list[1:]:
    props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0    
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

# Obtain rough single organoid outlines by fusing Cellpose labels and dilation, merging, erosion morphological operations
mip_labels, organoid_labels = segment_organoids_from_cp_labels(cytoplasm_labels)
viewer.add_labels(organoid_labels)

# Display final stats (including per cell and per organoid)
final_df = extract_organoid_stats_and_merge(mip_labels, organoid_labels, props_df)
final_df

Analyzing channel: Occludin_RFP in membrane ...
Analyzing channel: Claudin_FITC in membrane ...
Analyzing channel: Occludin_RFP in cell ...
Analyzing channel: DAPI in nuclei ...
Cells mapped to no organoid: 191 - 12.28% of total cells (1556)


,filename,well_id,organoid,label,membrane_area_x,membrane_Occludin_RFP_mean_int,membrane_Occludin_RFP_min_int,membrane_Occludin_RFP_max_int,membrane_Occludin_RFP_std_int,membrane_Occludin_RFP_max_mean_ratio,...,nuclei_DAPI_min_int,nuclei_DAPI_max_int,nuclei_DAPI_std_int,nuclei_DAPI_max_mean_ratio,nuclei_DAPI_sum_int,organoid_area,organoid_perimeter,organoid_eccentricity,organoid_solidity,organoid_extent
0,B2,B2,0,1,211.0,149.018957,108.0,234.0,21.120050,1.570270,...,366.0,1390.0,213.271758,2.016370,94442.0,NaN,NaN,NaN,NaN,NaN
1,B2,B2,0,2,485.0,165.012371,108.0,268.0,26.974976,1.624121,...,351.0,3756.0,680.632604,2.978715,707391.0,NaN,NaN,NaN,NaN,NaN
2,B2,B2,1,3,580.0,164.425862,101.0,277.0,24.150670,1.684650,...,374.0,2071.0,382.631708,2.195016,514208.0,6643.0,326.007143,0.462363,0.951174,0.793668
3,B2,B2,0,4,555.0,146.093694,109.0,201.0,15.278267,1.375829,...,289.0,2112.0,419.890176,2.112856,814670.0,NaN,NaN,NaN,NaN,NaN
4,B2,B2,0,5,205.0,136.770732,99.0,180.0,14.682364,1.316071,...,443.0,4095.0,906.785594,2.837823,382397.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,B2,B2,18,1551,72.0,262.583333,196.0,381.0,34.806908,1.450968,...,454.0,922.0,90.668802,1.437561,89791.0,35067.0,813.369624,0.680382,0.938398,0.734003
1320,B2,B2,0,1552,144.0,190.090278,140.0,271.0,23.980870,1.425638,...,301.0,879.0,133.392057,1.651326,287974.0,NaN,NaN,NaN,NaN,NaN
1321,B2,B2,0,1553,72.0,191.972222,148.0,260.0,25.379876,1.354363,...,324.0,738.0,89.036945,1.495715,133714.0,NaN,NaN,NaN,NaN,NaN
1322,B2,B2,0,1554,44.0,210.727273,145.0,346.0,36.005911,1.641933,...,294.0,960.0,134.574847,1.878022,179934.0,NaN,NaN,NaN,NaN,NaN
